# Création d´une courbe de niveau-volume avec PyQGIS

Ce Notebook Jupyter vous guide dans le processus d'utilisation de PyQGIS pour créer une courbe de niveau-volume à partir d'un DEM (MNE). Le notebook suit le même code que celui utilisé pour le tutoriel dans QGIS, mais peut s'exécuter sans l'interface QGIS.

## 1. Importer les packages 
Commençons par importer les packages nécessaires à PyQGIS.

In [ ]:
from qgis.core import (
     QgsApplication, 
     QgsProcessingFeedback,
     QgsRasterLayer,
     QgsVectorLayer,
     QgsField,
     QgsVectorFileWriter
)

from qgis.PyQt.QtCore import QVariant

`QgsApplication` est nécessaire pour exécuter QGIS et `QgsProcessingFeedback` iest nécessaire pour exécuter les outils de traitement. Nous avons besoin de `QgsRasterLayer` et `QgsVectorLayer` tpour travailler avec des couches matricielles et vectorielles respectivement. `QgsField` est nécessaire pour manipuler les champs dans les tables d'attributs. Nous utilisons  `QVariant` pour définir le type de données des champs. Nous avons besoin de `QgsVectorFileWriter` pour écrire la sortie dans un fichier `.csv`.

## 2. Initialiser QGIS sans interface
L'étape suivante consiste à démarrer QGIS sans l'interface.
Le `setPrefixPath` doit pointer vers votre environnement. Dans ce cas, il pointe vers mon dossier utilisateur et l'environnement `tutorials` peut être trouvé sous `Anaconda3/envs`.
Les deux autres lignes sont nécessaires pour initialiser QGIS. Lorsque vous exécutez le code, ignorez l'avertissement de dépréciation qui peut apparaître.

In [ ]:
QgsApplication.setPrefixPath('C:/Users/hansa/Anaconda3/envs/tutorials', True)
qgs = QgsApplication([], False)
qgs.initQgis()

## 3. Chargez la couche raster du DEM
Notre DEM est stocké dans un GeoPackage dans notre dossier `Data`. Définissons le dossier (nous en aurons besoin plus tard aussi, donc nous le mettons dans une variable séparée) et le jeu de données.

In [ ]:
projectPath = "./Data/"
inputRasterDEM = "GPKG:" + projectPath + "data_stagevolume.gpkg:DTM"

Maintenant nous pouvons ouvrir la couche raster DEM en utilisant `QgsRasterLayer`, qui a besoin du nom du fichier (`inputRasterDEM`), du nom de la couche `"DEM"` et nous ajoutons `"gdal"`, car c'est un format supporté par GDAL.

In [ ]:
demLayer = QgsRasterLayer(inputRasterDEM,"DEM","gdal")

## 4. Calculer les statistiques de la bande pour déterminer la plage et l'incrément
Nous devons d'abord déterminer la plage d'élévations. Avec `bandStatistics`, nous pouvons calculer des statistiques à partir de la couche matricielle. Comme nous avons un raster à une seule bande, nous utilisons `bandstatistics(1)` pour nous référer à la bande 1, la seule bande de notre raster. Nous pouvons maintenant utiliser `minimumValue` et `maximumValue` et imprimer les résultats. [Ici] (https://opensourceoptions.com/blog/pyqgis-get-raster-band-statistics/) vous trouvezez des informations sur les statistiques des bandes

In [ ]:
stats = demLayer.dataProvider().bandStatistics(1)
demMinimum = stats.minimumValue
demMaximum = stats.maximumValue
print("min:",demMinimum,"m")
print("max:",demMaximum,"m")

Quelle est la valeur moyenne du raster ? Essayez d'écrire le code dans le champ ci-dessous.

Au lieu de soustraire `demMinimum` de `demMaximum`, nous pouvons aussi obtenir directement la plage en utilisant `range`.

In [ ]:
demRange = stats.range
print("Elevation Difference:",demRange,"m")

L'étape suivante consiste à déterminer un incrément du niveau pour lequel nous voulons avoir le volume correspondant plus tard.
Utilisons un incrément de 10% de la plage.

In [ ]:
increment = demRange / 10.0
print("Increment:",increment)

## 5. Itération sur les incréments d'élévation et calcul des volumes correspondants
Maintenant nous pouvons initialiser l'itération sur l'incrément. Nous devons définir un compteur `i=0` et créer une liste vide (`dbfList`) pour les fichiers `.dbf` qui sont créés à chaque itération.

In [ ]:
i = 0
dbfList = []

Python ne peut pas itérer avec des "floats" (chars), donc nous définissons une fonction `frange` pour gérer cela :

In [ ]:
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step

Maintenant nous pouvons boucler sur la gamme d'élévation de `demMinimum` à `demMaximum` avec le `increment` comme taille de pas.

Pour utiliser les outils de traitement de QGIS, nous devons d'abord importer `QgsNativeAlgorithms` de `qgis.analysis`, les ajouter à l'application et importer `processing`.

Dans la boucle, nous créons d'abord le nom du fichier de sortie de la table `.dbf`.
Puis nous exécutons l'outil `rastersurfacevolume` en passant les variables correctes au dictionnaire. Rappelez-vous que nous obtenons le dictionnaire d'un outil de traitement à partir de l'historique de la *Boîte à outils de traitement* en exécutant d'abord l'outil.
Après avoir lancé l'outil, nous devons lire la table `.dbf` de sortie. Ensuite, nous créons une autre boucle sur les lignes (caractéristiques) dans le tableau : `for feature in dbfTable.getFeatures():`.
Pour chaque ligne, nous convertissons le volume en valeur absolue et de m3 en km3. Le résultat de ce calcul doit ensuite être stocké dans la table. Nous ajoutons deux nouveaux champs avec `addAtrributes` : *Niveau* et *VolAbsKm3*. Tous deux ont le type de données Double, indiqué par `QVariant.Double`. Après avoir défini les nouveaux champs, nous pouvons les ajouter à la table avec `updateFields()`. Ensuite, nous devons écrire les valeurs de `level` et `VolumeKm3` dans les champs correspondants. Avec `startEditing()`, nous passons en mode édition. Avec une autre boucle sur les caractéristiques, nous attribuons les nouvelles valeurs avec `updateFeature` et sauvegardons les changements avec `commitChanges()`.

A la fin de la boucle principale, nous ajoutons les fichiers `dbf` à la `dbfList` que nous avons initialisée auparavant.

In [ ]:
from qgis.analysis import QgsNativeAlgorithms

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

import processing

for level in frange(demMinimum,demMaximum + 1,increment):
    # Define the output table name
    outTable = projectPath +"volume" + str(round(level*100.0))
    outTableDbf = outTable + ".dbf"
    
    # Run the raster surface volume tool with the variables
    Tool = processing.run("native:rastersurfacevolume", {'INPUT':demLayer,
                                                         'BAND':1,
                                                         'LEVEL':level,
                                                         'METHOD':1,
                                                         'OUTPUT_HTML_FILE':'TEMPORARY_OUTPUT',
                                                         'OUTPUT_TABLE':outTable + ".shp"})
    
    # Read the table
    dbfTable = QgsVectorLayer(outTableDbf, outTable, "ogr")
    
    # Convert the volumes from m3 to km3
    for feature in dbfTable.getFeatures():
        VolumeKm3 = abs(feature["Volume"])/1000000000.0
    
    # Add the level and volume in km3 fields to the table
    pr = dbfTable.dataProvider()
    pr.addAttributes([QgsField("Level", QVariant.Double),QgsField("VolAbsKm3", QVariant.Double)])
    dbfTable.updateFields()
    dbfTable.startEditing()
    for f in dbfTable.getFeatures():
        f["Level"] = level
        f["VolAbsKm3"] = VolumeKm3
        dbfTable.updateFeature(f)
    dbfTable.commitChanges()
 
    dbfList.append(outTableDbf)

## 6. Fusionner tous les fichiers dbf dans un tableau de niveau-volume 
Maintenant, tous les fichiers `dbf` créés dans la boucle précédente peuvent être fusionnés en un seul tableau de niveua-volume. Nous utilisons l'outil de traitement `mergevectorlayers` et traitons les fichiers dans la `dbfList`. Notez que la sortie est un shapefile. Mais les fichiers shapefiles ont aussi un fichier `.dbf`.

In [ ]:
# Merge all dbf files into one
processing.run("native:mergevectorlayers", {'LAYERS':dbfList,
                                            'CRS':None,
                                            'OUTPUT':projectPath + 'stagevolume.shp'})

## 7. Tracer la courbe de niveau-volume avec Python

Le fichier `stagevolume.dbf` peut être ajouté à QGIS et utilisé pour tracer la courbe de niveau-volume avec le plugin Data Plotly comme décrit dans l'autre tutoriel. Ici cependant, nous voulons procéder à la création de la courbe sans l'interface graphique de QGIS. Il est donc préférable de convertir le fichier DBF au format CSV.

Tout d'abord, nous lisons le fichier DBF avec PyQGIS :

In [ ]:
vlayer = QgsVectorLayer(projectPath + 'stagevolume.dbf', "StageVolume", "ogr")

Avec `QgsVectorFileWriter.writeAsVectorFormat` nous pouvons convertir ce fichier au format CSV :

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(vlayer, "./Data/stagevolume.csv", "utf-8", vlayer.crs(), "CSV", layerOptions = ['GEOMETRY=AS_XYZ'])

Avec Pandas, nous pouvons lire le CSV et tracer un graphique linéaire :

In [ ]:
import pandas as pd
%matplotlib inline

df  = pd.read_csv("./Data/stagevolume.csv")
df.plot(kind='line',x='VolAbsKm3',y='Level', title='Stage Volume Curve', xlabel='Volume (km3)', ylabel='Level (m above sea level)', legend=False)

*Par __[Hans van der Kwast](http://www.linkedin.com/in/jvdkwast)__*<br>
*__[IHE Delft Institute for Water Education](http://www.un-ihe.org)__*<br>
*Twitter: @hansakwast*

*Ces matérials sont en libre accès, sous la license [CC By-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)*